In [1]:
import pandas as pd
import numpy as np
from pycaret.regression import *

In [2]:
df1 = pd.read_excel(r"C:\Users\User\OneDrive\Desktop\Player data\Dataset\merged_ipl_2016_2017.xlsx")
df2 = pd.read_excel(r"C:\Users\User\OneDrive\Desktop\Player data\Dataset\merged_ipl_2020_2021.xlsx")
df3 = pd.read_excel(r"C:\Users\User\OneDrive\Desktop\Player data\Dataset\merged_ipl_2023_2024.xlsx")

In [4]:
df = pd.concat([df1, df2, df3], ignore_index=True)
print(df)

               Player  Runs  Mat  Inns  NO    Avg   BF      SR  100  50  ...  \
0      AB de Villiers   903   25    25   4  43.00  570  158.42    1   7  ...   
1         Aaron Finch   692   26    25   3  31.45  479  144.47    0   7  ...   
2          Adam Milne     7    4     3   1   3.50   12   58.33    0   0  ...   
3          Adam Zampa     5   11     2   0   2.50    8   62.50    0   0  ...   
4         Aditya Tare    26    6     5   0   5.20   38   68.42    0   0  ...   
..                ...   ...  ...   ...  ..    ...  ...     ...  ...  ..  ...   
636        Yash Dhull    16    4     3   0   5.33   23   69.57    0   0  ...   
637       Yash Thakur     0    0     0   0   0.00    0    0.00    0   0  ...   
638  Yashasvi Jaiswal  1060   30    29   2  39.26  661  160.36    2   6  ...   
639     Yudhvir Singh    22    5     4   0   5.50   16  137.50    0   0  ...   
640  Yuzvendra Chahal     0    0     0   0   0.00    0    0.00    0   0  ...   

     bow_inns     Ov  bow_runs  Wkts  b

In [5]:
import numpy as np
import re

def convert_price(price):
    if isinstance(price, str):
        price = price.strip().lower()
        num_match = re.search(r'[\d.]+', price)
        if not num_match:
            return np.nan
        num = float(num_match.group())
        if 'crore' in price:
            return num * 1e7
        elif 'lakh' in price:
            return num * 1e5
    return np.nan


In [6]:
df['auction Price'] = df['auction Price'].apply(convert_price)

# Before dropping, check how many valid entries exist
print("Total rows:", len(df))
print("Valid auction prices:", df['auction Price'].notna().sum())
print("Invalid auction prices:", df['auction Price'].isna().sum())

Total rows: 641
Valid auction prices: 304
Invalid auction prices: 337


In [7]:
print(df[['auction Price']].head())

   auction Price
0            NaN
1     62000000.0
2            NaN
3            NaN
4      2000000.0


In [8]:
df_with_target = df[df["auction Price"].notnull()].copy()
print(df_with_target)

                Player  Runs  Mat  Inns  NO    Avg   BF      SR  100  50  ...  \
1          Aaron Finch   692   26    25   3  31.45  479  144.47    0   7  ...   
4          Aditya Tare    26    6     5   0   5.20   38   68.42    0   0  ...   
5       Ajinkya Rahane   862   30    30   4  33.15  702  122.79    0   8  ...   
6        Akshdeep Nath    20    5     4   0   5.00   28   71.43    0   0  ...   
8        Ambati Rayudu   425   18    17   1  26.56  356  119.38    0   3  ...   
..                 ...   ...  ...   ...  ..    ...  ...     ...  ...  ..  ...   
630  Wanindu Hasaranga    33    8     5   3  16.50   28  117.86    0   0  ...   
631  Washington Sundar    60    7     5   1  15.00   60  100.00    0   0  ...   
633         Will Jacks   230    8     8   1  32.86  131  175.57    1   1  ...   
637        Yash Thakur     0    0     0   0   0.00    0    0.00    0   0  ...   
640   Yuzvendra Chahal     0    0     0   0   0.00    0    0.00    0   0  ...   

     bow_inns     Ov  bow_r

In [9]:
df.columns

Index(['Player', 'Runs', 'Mat', 'Inns', 'NO', 'Avg', 'BF', 'SR', '100', '50',
       '4s', '6s', 'bow_mat', 'bow_inns', 'Ov', 'bow_runs', 'Wkts', 'bow_avg',
       'Econ', 'bow_sr', '4w', '5w', 'auction Price'],
      dtype='object')

In [10]:
def augment_data(df, n_augment=2, noise_level=0.05):
    augmented_rows = []
    numeric_cols = df.select_dtypes(include=np.number).columns.drop("auction Price")  # Exclude target

    for _ in range(n_augment):
        noisy_df = df.copy()
        for col in numeric_cols:
            noise = np.random.normal(0, noise_level * df[col].std(), size=df.shape[0])
            noisy_df[col] += noise
        augmented_rows.append(noisy_df)

    return pd.concat([df] + augmented_rows, ignore_index=True)
augmented_df = augment_data(df_with_target, n_augment=2)


In [11]:


print(f"Original rows with target: {len(df_with_target)}")
print(f"Rows after augmentation: {len(augmented_df)}")
print (augmented_df)

Original rows with target: 304
Rows after augmentation: 912
                Player        Runs        Mat       Inns        NO        Avg  \
0          Aaron Finch  692.000000  26.000000  25.000000  3.000000  31.450000   
1          Aditya Tare   26.000000   6.000000   5.000000  0.000000   5.200000   
2       Ajinkya Rahane  862.000000  30.000000  30.000000  4.000000  33.150000   
3        Akshdeep Nath   20.000000   5.000000   4.000000  0.000000   5.000000   
4        Ambati Rayudu  425.000000  18.000000  17.000000  1.000000  26.560000   
..                 ...         ...        ...        ...       ...        ...   
907  Wanindu Hasaranga   26.696692   9.020645   4.945775  3.244698  16.267685   
908  Washington Sundar   77.071765   6.900387   4.716560  0.956924  15.552203   
909         Will Jacks  209.697905   8.270519   8.038131  1.029912  32.313875   
910        Yash Thakur  -15.665464  -0.662595   0.300741  0.017054  -0.127884   
911   Yuzvendra Chahal   -8.884295  -1.048790   0

In [12]:
from pycaret.regression import *

regression_setup = setup(
    data=augmented_df,
    target='auction Price',
    session_id=42,
    
    use_gpu=False  
)

,Description,Value
0,Session id,42
1,Target,auction Price
2,Target type,Regression
3,Original data shape,"(912, 23)"
4,Transformed data shape,"(912, 23)"
5,Transformed train set shape,"(638, 23)"
6,Transformed test set shape,"(274, 23)"
7,Numeric features,21
8,Categorical features,1
9,Preprocess,True


In [ ]:
print(augmented_df.head())

In [13]:
best_model = compare_models()
print(best_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,6122673.6111,260020230773809.5625,14125321.9384,0.8388,0.4319,0.3698,0.0940
xgboost,Extreme Gradient Boosting,9127414.3500,351279516955443.1875,17247802.0500,0.7728,0.5541,0.5410,0.1130
lightgbm,Light Gradient Boosting Machine,10402196.7225,371975108108181.1875,18429704.4767,0.7708,0.5548,0.5638,0.1380
rf,Random Forest Regressor,10505392.2371,392107105450768.8750,18731115.3622,0.7501,0.5557,0.5901,0.2170
gbr,Gradient Boosting Regressor,11794988.4217,417722983883458.8750,19554636.9562,0.7337,0.6269,0.7081,0.1030
ada,AdaBoost Regressor,17072346.8532,523252002314988.1250,22263905.0821,0.6686,0.9667,1.6774,0.0490
dt,Decision Tree Regressor,9608680.5556,560383097718254.0000,22234538.9016,0.6340,0.6024,0.4575,0.0220
lasso,Lasso Regression,14946430.5526,588473748118146.5000,23815412.5345,0.6273,0.7230,0.8889,0.4960
llar,Lasso Least Angle Regression,14946491.7587,588476622420247.2500,23815475.5535,0.6273,0.7230,0.8889,0.0230
lr,Linear Regression,14946492.2720,588476605738847.2500,23815475.2352,0.6273,0.7230,0.8889,1.1070


ExtraTreesRegressor(n_jobs=-1, random_state=42)


In [14]:
final_model = finalize_model(best_model)

save_model(final_model, 'best_ipl_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Runs', 'Mat', 'Inns', 'NO', 'Avg',
                                              'BF', 'SR', '100', '50', '4s',
                                              '6s', 'bow_mat', 'bow_inns', 'Ov',
                                              'bow_runs', 'Wkts', 'bow_avg',
                                              'Econ', 'bow_sr', '4w', '5w'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['Player'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('rest_encoding',
                  TransformerWrapper(include=['Player'],
                                     transformer=TargetEncoder(cols=['Player'],
                                                               handle_missing='return_nan'))),
  

In [ ]:
df.columns

In [15]:
import pandas as pd
from pycaret.regression import load_model, predict_model

# 1. Load the trained model
model = load_model("best_ipl_model")

# 2. Get expected input columns
expected_columns = list(model.feature_names_in_)
print(" Model expects these columns:\n", expected_columns)

# 3. Load new player data
data = pd.read_excel(r"C:\Users\User\OneDrive\Desktop\Player data\Dataset\2023.csv.xlsx")

#  4. Strip and standardize input column names to avoid hidden mismatches
data.columns = data.columns.astype(str).str.strip()

# 5. Check for missing columns
missing_cols = [col for col in expected_columns if col not in data.columns]
if missing_cols:
    print(" Missing columns in input data:", missing_cols)

    
    print("\n Your data columns are:\n", list(data.columns))
else:
    print("All expected columns are present.")


Transformation Pipeline and Model Successfully Loaded
 Model expects these columns:
 ['Player', 'Runs', 'Mat', 'Inns', 'NO', 'Avg', 'BF', 'SR', '100', '50', '4s', '6s', 'bow_mat', 'bow_inns', 'Ov', 'bow_runs', 'Wkts', 'bow_avg', 'Econ', 'bow_sr', '4w', '5w', 'auction Price']
 Missing columns in input data: ['auction Price']

 Your data columns are:
 ['Player', 'Runs', 'Mat', 'Inns', 'NO', 'Avg', 'BF', 'SR', '100', '50', '4s', '6s', 'bow_mat', 'bow_inns', 'Ov', 'bow_runs', 'Wkts', 'bow_avg', 'Econ', 'bow_sr', '4w', '5w', 'prediction_label', 'prediction_label.1', 'prediction_label.2']


In [16]:
predictions = predict_model(model, data=data)
predictions.rename(columns={"Label": "Predicted Auction Price"}, inplace=True)



In [ ]:
# 7. Save the predictions to a new Excel file
predictions.to_excel(r"C:\Users\User\OneDrive\Desktop\Player data\Dataset\2023.csv.xlsx", index=False)
print("Prediction complete. Results saved to '2023_predictions.xlsx'")


In [ ]:
import pandas as pd

# Load your original file
m= pd.read_excel(r"C:\Users\User\OneDrive\Desktop\Player data\Dataset\2023.csv.xlsx")
m.to_excel(r"C:\Users\User\OneDrive\Desktop\Player data\Dataset\2023.csv.xlsx", index=False)

print("Cleaned and saved as '2023.xlsx'")


In [ ]:

print(df.dtypes[['100', '50']])


In [ ]:

print("📦 Your cleaned data columns:\n", m.columns.tolist())




In [ ]:
expected = set(model.feature_names_in_)
actual = set(df.columns)

missing = expected - actual
extra = actual - expected

if missing:
    print("❌ Missing columns in your data:", missing)
else:
    print("✅ All expected columns are present.")

if extra:
    print("ℹ️ Extra columns in your data (won’t be used):", extra)
